In [3]:
import numpy as np, pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

start_date = '2012-01-01 00:00:00'
#start_date = '2010-01-01 00:00:00'
end_date = '2018-12-31 00:00:00'
# Filter rows for a date range (e.g., from '2012-01-01' to '2018-12-31')

start_Day = 0
end_Day = 2531

df_covariates = pd.read_excel('../data/covariates_date.xlsx')

df = df_covariates.copy()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').set_index('Date')
# Optional: restrict to your analysis window
df = df.loc[start_date:end_date, ['Rain','WaterIndex','Temperature']].astype(float)


In [4]:
pearson = df[['Rain','WaterIndex']].corr(method='pearson').loc['Rain','WaterIndex']
spearman = df[['Rain','WaterIndex']].corr(method='spearman').loc['Rain','WaterIndex']
print('Pearson r:', round(pearson,3), ' | Spearman ρ:', round(spearman,3))


Pearson r: 0.73  | Spearman ρ: 0.81


In [5]:
def cross_corr(a, b, max_lag=30):
    a = (a - a.mean())/a.std()
    b = (b - b.mean())/b.std()
    lags = range(0, max_lag+1)  # rain at t-k vs flood at t
    vals = [a.shift(k).corr(b) for k in lags]
    best_k = int(np.nanargmax(np.abs(vals)))
    return pd.DataFrame({'lag_days': lags, 'ccf': vals}), lags[best_k], vals[best_k]

ccf_df, best_lag, best_ccf = cross_corr(df['Rain'], df['WaterIndex'], max_lag=30)
print(f'Peak cross-corr at +{best_lag} days: r = {best_ccf:.3f}')
# ccf_df is a table you can plot if needed


Peak cross-corr at +0 days: r = 0.730


In [6]:
LAGS = [0, 1, 7]  # example
X = pd.DataFrame(index=df.index)
for k in LAGS:
    X[f'Rain_l{k}']  = df['Rain'].shift(k)
    X[f'Flood_l{k}'] = df['WaterIndex'].shift(k)
X['Temp'] = df['Temperature']  # include if used
X = X.dropna()

# Standardize for conditioning diagnostics
Xs = pd.DataFrame(StandardScaler().fit_transform(X), index=X.index, columns=X.columns)

vifs = pd.Series([variance_inflation_factor(Xs.values, i) for i in range(Xs.shape[1])],
                 index=Xs.columns, name='VIF')
print(vifs.sort_values(ascending=False))


Rain_l1     852.029352
Rain_l0     680.425054
Flood_l1    194.606372
Flood_l0    160.392172
Rain_l7      22.944927
Flood_l7      8.466845
Temp          1.148224
Name: VIF, dtype: float64
